In [52]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt

import datetime
import os

In [53]:
IMG_SIZE = 128
BATH_SIZE = 32

In [54]:
train_datagen = ImageDataGenerator(
    rescale=1./255,         
    validation_split=0.2      
)

train_generator = train_datagen.flow_from_directory(
    'raw-img',           
    target_size=(IMG_SIZE, IMG_SIZE),     
    batch_size=BATH_SIZE,            
    class_mode='categorical',      
    subset='training'         
)

validation_generator = train_datagen.flow_from_directory(
    'raw-img',          
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATH_SIZE,
    class_mode='categorical',
    subset='validation'   
)

Found 7147 images belonging to 3 classes.
Found 1785 images belonging to 3 classes.


In [55]:
Repetitions = 5
TrainingModels = []
EpochList = [10, 15, 20, 25]

In [ ]:
def build_model(filtersList, denseUnits, shape, activation):
    
    model = models.Sequential()
    
    for filters in filtersList:
        model.add(layers.Conv2D(filters, (3, 3), activation=activation, input_shape=shape))
        model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    
    for dense_units in denseUnits:
        model.add(layers.Dense(dense_units, activation=activation))
    
    model.add(layers.Dense(3, activation='softmax'))

    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    return model


In [ ]:
TrainingModels.append(build_model([2, 4, 8, 16, 32, 64, 128], [512, 256, 128, 64, 32, 16, 8, 4]), (IMG_SIZE, IMG_SIZE, 3), 'relu')
TrainingModels.append(build_model([2, 4, 8, 16, 32, 64, 128], [256, 128, 64, 32, 16, 8, 4]),(IMG_SIZE, IMG_SIZE, 3), 'relu')
TrainingModels.append(build_model([2, 4, 8, 16, 32, 64, 128], [128, 64, 32, 16, 8, 4]),(IMG_SIZE, IMG_SIZE, 3), 'relu')
TrainingModels.append(build_model([2, 4, 8, 16, 32, 64, 128], [64, 32, 16, 8, 4]),(IMG_SIZE, IMG_SIZE, 3), 'relu')
TrainingModels.append(build_model([2, 4, 8, 16, 32, 64, 128], [32, 16, 8, 4]),(IMG_SIZE, IMG_SIZE, 3), 'relu')

TrainingModels.append(build_model([2, 4, 8, 16, 32, 64], [128, 64, 32, 16, 8, 4]),(IMG_SIZE, IMG_SIZE, 3), 'relu')
TrainingModels.append(build_model([2, 4, 8, 16, 32], [128, 64, 32, 16, 8, 4]),(IMG_SIZE, IMG_SIZE, 3), 'relu')
TrainingModels.append(build_model([2, 4, 8, 16], [128, 64, 32, 16, 8, 4]),(IMG_SIZE, IMG_SIZE, 3), 'relu')

In [ ]:
log_path = './training_log.txt'

if not os.path.exists(log_path):
    with open(log_path, 'w') as log_file:
        log_file.write("Timestamp,Model,Epochs,Repetition,Validation Loss,Validation Accuracy\n")

# Loop through training configurations
for model_index, model in enumerate(TrainingModels):
    for epoch in EpochList:
        for repetition in range(1, Repetitions + 1):
            print(f"Training Model {model_index + 1}, Epochs: {epoch}, Repetition: {repetition}")

            # Train the model
            history = model.fit(
                train_generator,
                validation_data=validation_generator,
                epochs=epoch,
                verbose=1  # Show progress
            )

            # Generate a timestamp
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

            # Evaluate the model
            validation_metrics = model.evaluate(validation_generator, verbose=1)
            val_loss, val_accuracy = validation_metrics[0], validation_metrics[1]

            # Save the model with timestamp
            model_path = f'./models/model_{model_index + 1}_epochs_{epoch}_rep_{repetition}_{timestamp}.keras'
            model.save(model_path)
            print(f"Model saved at {model_path}")

            # Log the results
            with open(log_path, 'a') as log_file:
                log_file.write(f"{timestamp},Model_{model_index + 1},{epoch},{repetition},{val_loss:.4f},{val_accuracy:.4f}\n")

            print(f"Logged results: Loss = {val_loss:.4f}, Accuracy = {val_accuracy:.4f}\n")

Training Model 1, Epochs: 10, Repetition: 1
Epoch 1/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 34s 146ms/step - accuracy: 0.5474 - loss: 0.9714 - val_accuracy: 0.5445 - val_loss: 0.8763
Epoch 2/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 33s 145ms/step - accuracy: 0.5620 - loss: 0.8675 - val_accuracy: 0.6213 - val_loss: 0.8272
Epoch 3/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 33s 146ms/step - accuracy: 0.6691 - loss: 0.7648 - val_accuracy: 0.7003 - val_loss: 0.6842
Epoch 4/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 33s 146ms/step - accuracy: 0.7086 - loss: 0.6479 - val_accuracy: 0.7300 - val_loss: 0.6372
Epoch 5/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 33s 146ms/step - accuracy: 0.7558 - loss: 0.5811 - val_accuracy: 0.7300 - val_loss: 0.6380
Epoch 6/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 33s 146ms/step - accuracy: 0.7933 - loss: 0.5176 - val_accuracy: 0.7927 - val_loss: 0.5588
Epoch 7/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 33s 145ms/step - accuracy: 0.8172 - loss: 0.4783 - val_accuracy: 0.7821 - val_loss: 0.5723
Epoch 8/10
224/224 ━━━━━━━━━━━━━━━━━━━━